In [1]:
import tensorflow as tf
import numpy as np
import sys
from my_load_data import load_dataset_fn
#from Prototype import Model

In [2]:
#Parameters of photo
total_num_batches = 124
img_size = 256
num_channels = 3
#Hyper parameters for network
num_classes = 2
batch_size = 10
total_steps = 2
learning_rate = 1e-4
result_dir = "./results"
"""Config variable named params, used for forwarding parameters through
whole model"""
#Params used as config file
params = {"result_dir":result_dir,
          "learning_rate_cl": learning_rate,
          "learning_rate_loc": learning_rate,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes,
          "batch_size":batch_size,
          "total_batches":total_num_batches,
          "total_steps":total_steps}

In [6]:
import tensorflow as tf
import functools
import os
import time 
import utils

def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator


class Model(object):
    """
    Model of neural network with all functionalities
    """
    def __init__(self, image, labels_class, labels_loc, params, mode=None):
        #Variables
        self.data = image
        self.target_class = labels_class
        self.target_loc = labels_loc
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.lr_cl = params["learning_rate_cl"]
        self.lr_loc = params["learning_rate_loc"]
        self.mode = mode
        self.global_step = tf.Variable(0, dtype=tf.int32,
                                       trainable=False, name='global_step')
        #Functions
        self.prediction
        self.classifier
        self.optimize_cl
        self.loss_cl
        #self.localizer
        #self.optimize_loc
        #self.loss_loc
        #self.summary
        
    @define_scope
    def prediction(self):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = input_layer, 
                                 filters = 16,
                                 kernel_size = 5,
                                 strides = 1,
                                 padding = "same",
                                 activation = tf.nn.relu)

        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #Resizing of pool1 output
        pool_1_flat = tf.reshape(pool1,[-1,int(self.img_size * self.img_size * 16 / 4)])

        #First Dense layer
        dense = tf.layers.dense(inputs = pool_1_flat,
                                units = 128,
                                activation = tf.nn.relu,
                                name = "Dense1")
        #Second Dense layer
        self.feature_map = tf.layers.dense(inputs = dense,
                                 units = 64,
                                 activation = tf.nn.relu,
                                 name = "Dense2")
        return self.feature_map
    ##################### CLASSIFIER ###############################
    @define_scope
    def classifier(self):
        """
        Assigns class to result
        """
        self.logits_cl = tf.layers.dense(inputs = self.feature_map,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)
        y_pred = tf.nn.softmax(logits=self.logits_cl)

        self.pred_cl = y_pred
        return self.pred_cl
    @define_scope
    def optimize_cl(self):
        """
        Optimizer of network, call after model.classifier to optimize classifier network
        """
        cross_entropy_cl =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_class, logits=self.pred_cl)
        self.opt_cl = tf.train.AdamOptimizer(self.lr_cl).minimize(cross_entropy_cl)
        return self.opt_cl
    @define_scope
    def loss_cl(self):
        """
        Function returning loss for classifier
        """
        self.loss_cl_val = tf.losses.softmax_cross_entropy(self.target_class, self.classifier)
        return self.loss_cl_val
    
    def accuracy_cl(self):
        """
        Count the number of right predictions in a batch
        """
        with tf.name_scope('accuracy')
            preds = tf.nn.softmax(self.logits_cl)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.target_class, 1))
            self.accuracy_cl_val = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
            return self.accuracy_cl_val
    
    #@define_scope
    def summary(self):
        """
        Create summaries to write on TensorBoard
        """
        with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss_cl_val)
            tf.summary.scalar('accuracy', self.accuracy_cl_val)
            tf.summary.histogram('histogram_loss', self.loss_cl_val)
            self.summary_op = tf.summary.merge_all()
            return self.summary_op
        
    
    
    def build_cl(self):
        """
        Build the computation graph
        """
        self.accuracy_cl()
        self.summary()

    def train_one_step_cl(self, sess, init, saver, writer, step, epoch):
        """
        Train one step of classifier
        """
        start_time = time.time()
        pass

##### Learning subroutine of network

In [11]:
def main(params, mode = None):
    total_steps = params["total_steps"]
    result_dir = params["result_dir"]
    #Assign name to graph
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(result_dir, graph=graph)
            
            #Placeholders for dataset and labels
            with tf.name_scope('Image'):
                ph_image = tf.placeholder(tf.float32, [None, params["img_size"], params["img_size"], params["num_channels"]])
            with tf.name_scope('Label_class'):
                ph_labels_class = tf.placeholder(tf.float32, [None, params["num_classes"]])
            with tf.name_scope('Label_localization'):
                ph_labels_loc = tf.placeholder(tf.float32, [None, 4])
                
            #Initialization of Model
            model = Model(ph_image, ph_labels_class, ph_labels_loc,
                          params, mode = None)
            model.build_cl()
            #Initialization of variables
            sess.run(tf.global_variables_initializer())
            print('Variables initialized')

            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            save_path=saver.save(sess, result_dir)
            print("Session saved to path:",save_path)
            
            #Training
            print("Starting Training")
            for step in range(1,total_steps+1):
                dataset, labels = load_dataset_fn(np.round(step/total_steps*params["total_batches"])+1)
                #
                feed_dict = {ph_image: dataset, 
                             ph_labels_class: labels[:, [0,1]], 
                             ph_labels_loc: labels[:,[2,3,4,5]]}
                
                del dataset, labels
                
                #########SESS.RUN HERE###############
                loss, _, summary = sess.run([model.loss_cl, model.optimize_cl, model.summary_op], 
                                            feed_dict= feed_dict)
                
                writer.add_summary(summary, global_step=step)
                #Printing data every 10th step
                if (step % 1 == 0):
                    print('Loss at step %d: %f' % (step, loss))
                    #print("Accuracy: {0:2.2f}".format(accuracy(pred, feed_dict[ph_labels_class])))
                if (step + 1) % 1000==0:
                    saver.save(sess, 'checkpoint_directory/model_name',
                               global_step=model.global_step)
        
        
        print("Finnished session")
        #Merge all summaries
        #writer.flush()
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")
        



In [12]:
tf.reset_default_graph()
if __name__ == '__main__':
     main(params)

Starting Session
Initialized Model.prediction
Initialized Model.classifier
Initialized Model.optimize_cl
Initialized Model.loss_cl
Variables initialized
Session saved to path: ./results
Starting Training
Loss at step 1: 0.693490
Loss at step 2: 0.695700
Finnished session
Closed summary, work finnished


In [ ]:
###Paste into conda prompt
#   tensorboard --logdir="results/"
#   tensorboard --logdir="pythondata/MITP Project/results/"

####################### LOCALIZER ################################

    @define_scope
    def localizer(self):
        """
        Function tries to obtain localization of marker on photo
        """
        #Conversion to num_classes output size
        logits_loc = tf.layers.dense(inputs = self.prediction,
                                 units = 4,
                                 activation = tf.nn.relu)
        
        # Softmax output of the neural network
        self.pred_loc = tf.nn.softmax(logits=logits_loc)
        
    
    @define_scope
    def optimize_loc(self):
        """
        Optimizes localizer
        """
        cross_entropy_loc =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_loc, logits=self.pred_loc)
        self.opt_loc = tf.train.AdamOptimizer(self.lr_loc).minimize(cross_entropy_loc)
    
    @define_scope
    def loss_loc(self):
        """
        Returns loss for localizator
        """
        loss_loc = tf.losses.mean_squared_error(self.target_loc, self.pred_loc)
    
    
    
    
    def train(self, n_epochs):
        '''
        The train function alternates between training one epoch and evaluating
        '''
        utils.safe_mkdir('checkpoints')
        utils.safe_mkdir('checkpoints/convnet_layers')
        writer = tf.summary.FileWriter('./graphs/convnet_layers', tf.get_default_graph())

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_layers/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
            
            step = self.gstep.eval()

            for epoch in range(n_epochs):
                step = self.train_one_epoch(sess, saver, self.train_init, writer, epoch, step)
                self.eval_once(sess, self.test_init, writer, epoch, step)
        writer.close()